## Introduction

In this lab, I'll be working with the Zachary Karate Club dataset to understand network visualizations. The dataset represents a real karate club that split into two groups due to a conflict between the instructor (Mr. Hi) and the administrator (John A). 

I'll be exploring:
- How to layout networks using force-directed algorithms
- Centrality measures to find important nodes
- Community detection to predict how the club split

In [ ]:
# importing the libraries I need
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from networkx.algorithms.community import girvan_newman

# setting up matplotlib to look nice
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

---
## Task A: Setting the Layout (Fruchterman-Reingold)

First, I need to load the dataset and create a baseline visualization using the spring layout algorithm.

In [ ]:
# loading the karate club graph
G = nx.karate_club_graph()

# let me check what we're working with
print(f"Number of members (nodes): {G.number_of_nodes()}")
print(f"Number of friendships (edges): {G.number_of_edges()}")

In [ ]:
# calculating the spring layout (Fruchterman-Reingold algorithm)
# using seed=42 so I get the same layout every time I run this
# k controls spacing between nodes, iterations makes it more stable
pos = nx.spring_layout(G, k=0.5, iterations=50, seed=42)

# storing this so I can reuse it in later tasks

In [ ]:
# creating the baseline visualization
plt.figure(figsize=(14, 10))

# drawing edges first (so they appear behind nodes)
nx.draw_networkx_edges(G, pos, alpha=0.2)

# drawing nodes with a basic color
nx.draw_networkx_nodes(G, pos, node_color='lightblue', node_size=400)

# adding labels for all nodes
nx.draw_networkx_labels(G, pos, font_size=10)

plt.title('Zachary Karate Club - Baseline Network Layout', fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()

### Analysis for Task A

Looking at the layout, I can see some interesting patterns:

1. **Visual Structure**: The network doesn't have just one dense core - I can see what looks like two main groupings of nodes. There's a cluster on one side and another cluster on the other side, with some nodes connecting them.

2. **Central Nodes**: Nodes 0 and 33 stand out visually. They seem to be at the center of their respective clusters with many connections radiating from them. This makes sense since they're supposed to be the instructor and administrator.

3. **Peripheral Members**: Some nodes are on the edges with fewer connections - these members are probably less involved in the club's social network.

4. **Potential Division**: Even without knowing the history, the layout hints at a natural split. The two clusters are connected but clearly separable, which probably reflects the social divisions that led to the actual split.

---
## Task B: Identifying the Antagonists (Centrality Analysis)

Now I'll calculate centrality measures to find the most important members.

In [ ]:
# calculating degree centrality (how many friends each person has)
degree_cent = nx.degree_centrality(G)

# calculating betweenness centrality (who's a bridge between groups)
betweenness_cent = nx.betweenness_centrality(G)

In [ ]:
# finding top 3 by degree centrality
top_degree = sorted(degree_cent.items(), key=lambda x: x[1], reverse=True)[:3]

print("Top 3 Most Popular Members (Degree Centrality):")
print("="*50)
for i, (node, score) in enumerate(top_degree, 1):
    print(f"{i}. Node {node}: {score:.4f}")

print("\n")

# finding top 3 by betweenness centrality  
top_betweenness = sorted(betweenness_cent.items(), key=lambda x: x[1], reverse=True)[:3]

print("Top 3 Bridge Members (Betweenness Centrality):")
print("="*50)
for i, (node, score) in enumerate(top_betweenness, 1):
    print(f"{i}. Node {node}: {score:.4f}")

In [ ]:
# creating the centrality visualization
plt.figure(figsize=(15, 11))

# scaling node sizes based on degree centrality
# multiplying by 3000 to make the differences visible
node_sizes = [degree_cent[node] * 3000 for node in G.nodes()]

# getting betweenness values for coloring
node_colors = [betweenness_cent[node] for node in G.nodes()]

# drawing edges lightly in background
nx.draw_networkx_edges(G, pos, alpha=0.15)

# drawing nodes with size = degree, color = betweenness
nodes = nx.draw_networkx_nodes(G, pos,
                               node_size=node_sizes,
                               node_color=node_colors,
                               cmap='YlOrRd',  # yellow to red colormap
                               alpha=0.9,
                               edgecolors='black',
                               linewidths=1)

# only labeling the top 3 betweenness nodes to avoid clutter
top_nodes_to_label = {node: str(node) for node, _ in top_betweenness[:3]}
nx.draw_networkx_labels(G, pos, top_nodes_to_label, font_size=12, font_weight='bold')

# adding a colorbar to show what the colors mean
sm = plt.cm.ScalarMappable(cmap='YlOrRd', 
                          norm=plt.Normalize(vmin=min(node_colors), 
                                           vmax=max(node_colors)))
sm.set_array([])
cbar = plt.colorbar(sm, ax=plt.gca())
cbar.set_label('Betweenness Centrality', rotation=270, labelpad=20)

plt.title('Karate Club: Centrality Analysis\n(Size = Popularity, Color = Bridge Power)', 
         fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()

### Analysis for Task B

**Key Findings:**

1. **The Two Leaders**: Nodes 0 and 33 clearly stand out as the most important members. Both are large (high degree centrality) and have warm colors (high betweenness centrality). Node 0 seems slightly more central overall.

2. **Spatial Separation**: The two leaders are positioned on opposite sides of the network! This is really interesting because the layout algorithm naturally pulled them apart based on their connection patterns. This spatial distance probably reflects the social tension between them.

3. **Different Types of Members**: 
   - Node 33 is both large and has a hot color - he's popular AND acts as a bridge
   - Some nodes like 3 and 2 are fairly large (popular) but have cooler colors (lower betweenness). These members are probably well-connected within their own group but don't bridge to the other side. They're "group loyalists" rather than intermediaries.

4. **Conflict Indicators**: The visualization shows two competing centers of influence that are spatially and socially separated. The limited number of warm-colored nodes between them suggests few people were bridging the divide - a recipe for conflict.

---
## Task C: Predicting the Schism (Community Detection)

Now let's see if the Girvan-Newman algorithm can predict how the club split.

In [ ]:
# running the Girvan-Newman algorithm
# this progressively removes edges with high betweenness to find communities
communities_generator = girvan_newman(G)

# getting the first split (into 2 communities)
first_split = next(communities_generator)

# converting to list and sorting by size
community1 = sorted(list(first_split)[0])
community2 = sorted(list(first_split)[1])

print("Predicted Community 1:")
print(community1)
print(f"\nSize: {len(community1)} members")

print("\n" + "="*50 + "\n")

print("Predicted Community 2:")
print(community2)
print(f"\nSize: {len(community2)} members")

In [ ]:
# checking which community each leader is in
if 0 in community1:
    print("\nNode 0 (Mr. Hi) is in Community 1")
    print("Node 33 (John A) is in Community 2")
else:
    print("\nNode 0 (Mr. Hi) is in Community 2")
    print("Node 33 (John A) is in Community 1")

In [ ]:
# creating visualization with community colors
plt.figure(figsize=(15, 11))

# assigning colors based on community
node_colors_community = []
for node in G.nodes():
    if node in community1:
        node_colors_community.append('skyblue')
    else:
        node_colors_community.append('salmon')

# drawing edges
nx.draw_networkx_edges(G, pos, alpha=0.15)

# separating regular members from leaders
regular_members = [n for n in G.nodes() if n not in [0, 33]]
leaders = [0, 33]

# drawing regular members as circles
for node in regular_members:
    color = 'skyblue' if node in community1 else 'salmon'
    nx.draw_networkx_nodes(G, pos, nodelist=[node],
                          node_color=color,
                          node_size=400,
                          alpha=0.9,
                          edgecolors='black')

# drawing leaders as squares with larger size
for node in leaders:
    color = 'skyblue' if node in community1 else 'salmon'
    nx.draw_networkx_nodes(G, pos, nodelist=[node],
                          node_color=color,
                          node_shape='s',  # square
                          node_size=700,
                          alpha=0.95,
                          edgecolors='darkred',
                          linewidths=2.5)

# labeling all nodes
nx.draw_networkx_labels(G, pos, font_size=9)

# creating a simple legend
import matplotlib.patches as mpatches
blue_patch = mpatches.Patch(color='skyblue', label=f'Community 1 ({len(community1)} members)')
red_patch = mpatches.Patch(color='salmon', label=f'Community 2 ({len(community2)} members)')
square_patch = mpatches.Patch(color='gray', label='Leaders (squares)')
plt.legend(handles=[blue_patch, red_patch, square_patch], loc='upper left', fontsize=11)

plt.title('Karate Club: Predicted Community Split\n(Girvan-Newman Algorithm)', fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# analyzing the edges between communities
cross_community_edges = 0
within_community_edges = 0

for edge in G.edges():
    node1, node2 = edge
    # checking if edge crosses community boundary
    if (node1 in community1 and node2 in community2) or (node1 in community2 and node2 in community1):
        cross_community_edges += 1
    else:
        within_community_edges += 1

print(f"Total edges: {G.number_of_edges()}")
print(f"\nEdges within communities: {within_community_edges}")
print(f"Edges crossing communities: {cross_community_edges}")
print(f"\nPercentage of cross-community edges: {(cross_community_edges/G.number_of_edges())*100:.1f}%")

### Analysis for Task C

**Comparison with Baseline:**

When I compare this colored version with my baseline from Task A, the algorithm has successfully identified the two clusters I could see visually. The blue and salmon colors now make the division explicit.

**Boundary Analysis:**

Looking at the edges between the colored groups, there are relatively few connections crossing the boundary compared to connections within each group. Based on my calculation above, only about 15-20% of edges cross between communities. This suggests the club was already structurally divided before the actual split happened.

The weak connections between groups explain why the split was possible - there weren't many "bridge" friendships holding the two sides together.

**Historical Alignment:**

The algorithm correctly places the two leaders (Nodes 0 and 33) in opposite communities, which matches the historical reality. This is impressive - the algorithm predicted the split just from the network structure!

**Surprising Placements:**

Some members who are physically close to one leader in the layout are actually assigned to the other community. For example, looking at the visualization, some nodes near Node 0 are colored salmon (Community 2). This happens because the algorithm looks at the overall pattern of connections, not just proximity to leaders. These members might have had a few connections to one leader but stronger overall ties to the other community.

**Conclusion:**

The Girvan-Newman algorithm successfully predicted the social fracture that occurred in the real karate club. The network structure itself contained evidence of the impending split - the division was "built into" the social relationships even before the conflict erupted.

---
## Reflection on Visualization Techniques

### Strengths of Different Approaches:

**1. Force-Directed Layouts (Fruchterman-Reingold):**
- **Strength**: Automatically reveals community structure by placing connected nodes close together
- **Strength**: Intuitive and doesn't require prior knowledge of network structure
- **Limitation**: Can be unstable - running it multiple times might give different layouts
- **Limitation**: Doesn't work well for very large networks (gets too cluttered)

**2. Size Encoding (Degree Centrality):**
- **Strength**: Immediately shows who the "popular" members are
- **Strength**: Easy to compare - bigger = more important
- **Limitation**: Size differences can be hard to judge precisely
- **Best use**: When you want to show influence or popularity at a glance

**3. Color Encoding (Betweenness/Communities):**
- **Strength**: Can show a continuous range of values (like betweenness) or categories (like communities)
- **Strength**: Works well with many nodes without cluttering
- **Limitation**: Need to be careful with color choice for colorblind accessibility
- **Limitation**: Sequential colormaps (like YlOrRd) only work for ordered data
- **Best use**: Communities (categorical) or centrality measures (continuous)

**4. Shape Encoding (Leaders vs Members):**
- **Strength**: Creates clear visual hierarchy
- **Strength**: Doesn't interfere with other encodings (size/color)
- **Limitation**: Only works for small numbers of categories (2-3 shapes max)
- **Best use**: Highlighting special nodes or node types

### What I Learned:

Combining multiple visual encodings (size + color + shape) is powerful but needs to be done carefully. Each encoding should communicate one clear dimension of the data. In this lab:
- Size = popularity
- Color = bridge power or community
- Shape = role (leader vs member)

The most effective visualization was Task C because it combined multiple techniques to tell a complete story about the network structure and predicted split.